In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.models import load_model, Model
from keras.callbacks import EarlyStopping
import keras.backend as K
import numpy as np

# our stuff
from synth import *
from utils import *
from const import Tx, Ty, n_a, n_s, input_vocab, output_vocab, output_vocab_inv

import matplotlib.pyplot as plt
%matplotlib inline

/home/opyate/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# check GPU
from tensorflow.python.client import device_lib
[print(x.name) for x in device_lib.list_local_devices()]

/device:CPU:0
/device:GPU:0


[None, None]

In [3]:
# Returns input,output,dt
# Run once to see an example.
load_time()

("five minutes and twenty nine seconds to eleven o'clock at night",
 '22:54:31',
 datetime.time(22, 54, 31))

In [4]:
# One-hot representations of load_time()'s input and output
# will be fed into the model.

i, o, _ = load_time()

print('Input:', i)
print('Target:', o)

ioh, ooh = one_hot([i], [o], Tx, Ty, input_vocab, output_vocab)

print('Input shape:', ioh.shape)
print('Target shape:', ooh.shape)
print(*ooh)

Input: twenty three minutes and twenty one seconds past four o'clock in the morning
Target: 04:23:21
Input shape: (1, 128, 102)
Target shape: (1, 8, 11)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


- ioh, "input one-hot", has shape (128,102), i.e. it can accept an input string of up to (or padded to) 128 characters of a possible vocabulary of 102 characters.
- ooh, "output one-hot", has shape (8,11), i.e. the model will predict 8 characters of the form 'hh:mm:ss' of a possible vocabulary of all digits and colon ':'.

In [5]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
# softmax(axis = 1) from utils.py
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [6]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e.
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas"
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas, a])
    
    return context

In [7]:
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(output_vocab), activation=softmax)

In [8]:
def get_model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # input of shape (Tx,)
    X = Input(shape=(Tx, human_vocab_size))
    
    # s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    
    s = s0
    c = c0
    
    outputs = []
    
    # pre-attention Bi-LSTM
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    for t in range(Ty):
    
        # perform one step of the attention mechanism to get back the context vector at step t
        context = one_step_attention(a, s)
        
        # apply the post-attention LSTM cell to the "context" vector.
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(s)
        
        outputs.append(out)
    
    # create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    return model

In [9]:
model = get_model(Tx, Ty, n_a, n_s, len(input_vocab), len(output_vocab))

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 102)     0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 128, 128)     85504       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 128, 128)     0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

In [11]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=2, verbose=0, mode='auto')
data_maker = dataset_generator(Tx, Ty, input_vocab, output_vocab, n_s, batch_size=1000)
validation_maker = dataset_generator(Tx, Ty, input_vocab, output_vocab, n_s, batch_size=1000)
model.fit_generator(data_maker, validation_data=validation_maker, validation_steps=100, steps_per_epoch=1000, epochs=1000, callbacks=[early_stop])

Epoch 1/1000
1000/1000 [==============================] - 249s 249ms/step - loss: 7.4680 - dense_2_loss_1: 0.5685 - dense_2_loss_2: 1.8177 - dense_2_loss_3: 0.0662 - dense_2_loss_4: 0.7530 - dense_2_loss_5: 1.3919 - dense_2_loss_6: 0.0893 - dense_2_loss_7: 1.1506 - dense_2_loss_8: 1.6309 - dense_2_acc_1: 0.7456 - dense_2_acc_2: 0.2887 - dense_2_acc_3: 0.9990 - dense_2_acc_4: 0.6544 - dense_2_acc_5: 0.4214 - dense_2_acc_6: 0.9990 - dense_2_acc_7: 0.5162 - dense_2_acc_8: 0.3718 - val_loss: 3.9772 - val_dense_2_loss_1: 0.1837 - val_dense_2_loss_2: 1.2240 - val_dense_2_loss_3: 0.0010 - val_dense_2_loss_4: 0.3723 - val_dense_2_loss_5: 0.7840 - val_dense_2_loss_6: 0.0025 - val_dense_2_loss_7: 0.5519 - val_dense_2_loss_8: 0.8578 - val_dense_2_acc_1: 0.9379 - val_dense_2_acc_2: 0.5170 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 0.8308 - val_dense_2_acc_5: 0.6563 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 0.7843 - val_dense_2_acc_8: 0.6907
Epoch 2/1000
1000/1000 [====================

1000/1000 [==============================] - 246s 246ms/step - loss: 0.0627 - dense_2_loss_1: 0.0018 - dense_2_loss_2: 0.0143 - dense_2_loss_3: 7.6454e-05 - dense_2_loss_4: 0.0062 - dense_2_loss_5: 0.0205 - dense_2_loss_6: 1.1466e-04 - dense_2_loss_7: 0.0103 - dense_2_loss_8: 0.0093 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9993 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 0.9993 - dense_2_acc_5: 0.9953 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 0.9992 - dense_2_acc_8: 0.9997 - val_loss: 0.0534 - val_dense_2_loss_1: 0.0016 - val_dense_2_loss_2: 0.0122 - val_dense_2_loss_3: 6.2462e-05 - val_dense_2_loss_4: 0.0054 - val_dense_2_loss_5: 0.0177 - val_dense_2_loss_6: 1.0030e-04 - val_dense_2_loss_7: 0.0088 - val_dense_2_loss_8: 0.0075 - val_dense_2_acc_1: 1.0000 - val_dense_2_acc_2: 0.9995 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 0.9993 - val_dense_2_acc_5: 0.9964 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 0.9993 - val_dense_2_acc_8: 0.9999
Epoch 10/1000
1000/1000 [================

1000/1000 [==============================] - 245s 245ms/step - loss: 0.0114 - dense_2_loss_1: 5.1393e-04 - dense_2_loss_2: 0.0033 - dense_2_loss_3: 7.6580e-06 - dense_2_loss_4: 0.0011 - dense_2_loss_5: 0.0034 - dense_2_loss_6: 1.4462e-05 - dense_2_loss_7: 0.0017 - dense_2_loss_8: 0.0014 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9997 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 0.9999 - dense_2_acc_5: 0.9999 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - val_loss: 0.0105 - val_dense_2_loss_1: 5.2986e-04 - val_dense_2_loss_2: 0.0032 - val_dense_2_loss_3: 6.7855e-06 - val_dense_2_loss_4: 8.9261e-04 - val_dense_2_loss_5: 0.0031 - val_dense_2_loss_6: 1.2488e-05 - val_dense_2_loss_7: 0.0014 - val_dense_2_loss_8: 0.0013 - val_dense_2_acc_1: 0.9999 - val_dense_2_acc_2: 0.9997 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 1.0000 - val_dense_2_acc_5: 1.0000 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 1.0000 - val_dense_2_acc_8: 1.0000
Epoch 18/1000
1000/1000 [====

1000/1000 [==============================] - 246s 246ms/step - loss: 0.0045 - dense_2_loss_1: 3.3013e-04 - dense_2_loss_2: 0.0017 - dense_2_loss_3: 1.5270e-06 - dense_2_loss_4: 3.5974e-04 - dense_2_loss_5: 0.0011 - dense_2_loss_6: 3.2074e-06 - dense_2_loss_7: 5.4418e-04 - dense_2_loss_8: 4.9559e-04 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9997 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - val_loss: 0.0043 - val_dense_2_loss_1: 3.0702e-04 - val_dense_2_loss_2: 0.0017 - val_dense_2_loss_3: 1.4040e-06 - val_dense_2_loss_4: 3.4709e-04 - val_dense_2_loss_5: 0.0010 - val_dense_2_loss_6: 3.1554e-06 - val_dense_2_loss_7: 4.9766e-04 - val_dense_2_loss_8: 4.5854e-04 - val_dense_2_acc_1: 0.9999 - val_dense_2_acc_2: 0.9996 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 1.0000 - val_dense_2_acc_5: 1.0000 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 1.0000 - val_dense_2_acc_8: 1.0000
Epoch 26/

1000/1000 [==============================] - 245s 245ms/step - loss: 0.0029 - dense_2_loss_1: 2.1230e-04 - dense_2_loss_2: 0.0010 - dense_2_loss_3: 5.6552e-07 - dense_2_loss_4: 1.6669e-04 - dense_2_loss_5: 4.9267e-04 - dense_2_loss_6: 1.2964e-06 - dense_2_loss_7: 5.1637e-04 - dense_2_loss_8: 4.3676e-04 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9998 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 0.9999 - dense_2_acc_8: 0.9999 - val_loss: 0.0027 - val_dense_2_loss_1: 3.3717e-04 - val_dense_2_loss_2: 0.0012 - val_dense_2_loss_3: 4.7762e-07 - val_dense_2_loss_4: 1.5979e-04 - val_dense_2_loss_5: 5.1907e-04 - val_dense_2_loss_6: 1.5057e-06 - val_dense_2_loss_7: 2.7978e-04 - val_dense_2_loss_8: 2.7953e-04 - val_dense_2_acc_1: 0.9999 - val_dense_2_acc_2: 0.9998 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 1.0000 - val_dense_2_acc_5: 1.0000 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 1.0000 - val_dense_2_acc_8: 1.0000
E

1000/1000 [==============================] - 246s 246ms/step - loss: 0.0017 - dense_2_loss_1: 1.9977e-04 - dense_2_loss_2: 7.8147e-04 - dense_2_loss_3: 3.3130e-07 - dense_2_loss_4: 1.0890e-04 - dense_2_loss_5: 3.0797e-04 - dense_2_loss_6: 7.7325e-07 - dense_2_loss_7: 1.5713e-04 - dense_2_loss_8: 1.4308e-04 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9998 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - val_loss: 0.0018 - val_dense_2_loss_1: 1.9727e-04 - val_dense_2_loss_2: 8.5911e-04 - val_dense_2_loss_3: 3.1898e-07 - val_dense_2_loss_4: 1.0732e-04 - val_dense_2_loss_5: 2.9617e-04 - val_dense_2_loss_6: 9.0698e-07 - val_dense_2_loss_7: 1.5501e-04 - val_dense_2_loss_8: 1.4391e-04 - val_dense_2_acc_1: 1.0000 - val_dense_2_acc_2: 0.9998 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 1.0000 - val_dense_2_acc_5: 1.0000 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 1.0000 - val_dense_2_acc_8: 

1000/1000 [==============================] - 245s 245ms/step - loss: 0.0013 - dense_2_loss_1: 1.5340e-04 - dense_2_loss_2: 6.8492e-04 - dense_2_loss_3: 2.5861e-07 - dense_2_loss_4: 7.5033e-05 - dense_2_loss_5: 2.1013e-04 - dense_2_loss_6: 6.1940e-07 - dense_2_loss_7: 1.0858e-04 - dense_2_loss_8: 1.0045e-04 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9998 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - val_loss: 0.0014 - val_dense_2_loss_1: 1.4814e-04 - val_dense_2_loss_2: 7.1452e-04 - val_dense_2_loss_3: 2.5350e-07 - val_dense_2_loss_4: 8.2569e-05 - val_dense_2_loss_5: 2.0853e-04 - val_dense_2_loss_6: 5.4994e-07 - val_dense_2_loss_7: 1.0670e-04 - val_dense_2_loss_8: 9.8614e-05 - val_dense_2_acc_1: 0.9999 - val_dense_2_acc_2: 0.9998 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 1.0000 - val_dense_2_acc_5: 1.0000 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 1.0000 - val_dense_2_acc_8: 

1000/1000 [==============================] - 245s 245ms/step - loss: 0.0011 - dense_2_loss_1: 1.5849e-04 - dense_2_loss_2: 5.8636e-04 - dense_2_loss_3: 2.2275e-07 - dense_2_loss_4: 5.4060e-05 - dense_2_loss_5: 1.4868e-04 - dense_2_loss_6: 4.6645e-07 - dense_2_loss_7: 7.6099e-05 - dense_2_loss_8: 6.8733e-05 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9999 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - val_loss: 9.9344e-04 - val_dense_2_loss_1: 1.3859e-04 - val_dense_2_loss_2: 5.1453e-04 - val_dense_2_loss_3: 2.1772e-07 - val_dense_2_loss_4: 5.3200e-05 - val_dense_2_loss_5: 1.4271e-04 - val_dense_2_loss_6: 4.1483e-07 - val_dense_2_loss_7: 7.6132e-05 - val_dense_2_loss_8: 6.7642e-05 - val_dense_2_acc_1: 0.9999 - val_dense_2_acc_2: 0.9999 - val_dense_2_acc_3: 1.0000 - val_dense_2_acc_4: 1.0000 - val_dense_2_acc_5: 1.0000 - val_dense_2_acc_6: 1.0000 - val_dense_2_acc_7: 1.0000 - val_dense_2_acc

In [13]:
EXAMPLES = [
    'noon',
    'five seconds past noon',
    "four o'clock in the morning",
    "half past eleven o'clock in the morning",
    "twenty nine minutes and fifty nine seconds past noon",
    "twenty nine minutes and fifty nine seconds past eleven o'clock at night",
    "twenty eight minutes to midnight"
]

for example in EXAMPLES:
    source = string_to_int(example, Tx, input_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(input_vocab)), source)))

    prediction = model.predict([np.expand_dims(source, axis=0), np.zeros((1,n_s)), np.zeros((1,n_s))])
    prediction = np.argmax(prediction, axis = -1)
    output = [output_vocab_inv[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))
    

source: noon
output: 02:00:00
source: five seconds past noon
output: 12:00:05
source: four o'clock in the morning
output: 04:00:00
source: half past eleven o'clock in the morning
output: 11:30:00
source: twenty nine minutes and fifty nine seconds past noon
output: 12:29:59
source: twenty nine minutes and fifty nine seconds past eleven o'clock at night
output: 23:29:59
source: twenty eight minutes to midnight
output: 23:32:00


In [14]:
# happy?

import os

modelfile = "themodel.h5"
if os.path.isfile(modelfile):
    print('already exists', modelfile)
else:
    print('saving', modelfile)
    model.save(modelfile)

modelweightsfile = "themodel-weights.h5"
if os.path.isfile(modelweightsfile):
    print('already exists', modelweightsfile)
else:
    print('saving', modelweightsfile)
    model.save_weights(modelweightsfile)


saving themodel.h5


/home/opyate/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2361: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'c0:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '
/home/opyate/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2361: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '
/home/opyate/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2361: UserWarning: Laye

saving themodel-weights.h5


See `attention.ipynb` for the attention map.